# Distributing your FloPy Plugin as a Python Package

FloPy plug-ins can be installed as python packages.  FloPy will automatically detect plug-ins in other python packages by using python package metadata.

In this notebook you will package an existing FloPy plug-in in its own python package and then run an example simulation using the plug-in in the python package.  The plug-in being packaged is generated in the flopy3_dev_mf6_BuildStressPlugin notebook. Please run that notebook first.

First, you need to determine the plug-in extension your FloPy plug-in is using and the package type of your FloPy plug-in's  package file.  These are necessary to identify the files you wish to export.  Also, define the type of model your plug-in is used in and optionally the name of the python package you wish to create (if you do not specify a name, a name will be created for you). 

In [ ]:
plugin_ext = "npc"
package_type = "fp_npc"
model_type = "gwf"
python_package_name = "fp_npc_plugin"

Next, run the create_python_package script.

In [ ]:
import flopy

flopy.mf6.utils.flopy_plugins.plugin_template.create_python_package(
    plugin_ext,
    package_type,
    model_type,
    python_package_name)

The create_python_package method creates a folder with all of your FloPy plug-in files and a setup.py file to install your plug-in.  You can created the plug-in python package ready to be installed with the following command:

python setup.py install

You may need to copy your new python package into your active python distribution's site-packages folder.  Once there, FloPy will automatically find and use the corresponding plug-in code when you run the simulation. Note that if you have a plugin with the same name in your FloPy installation (flopy/mf6/utils/flopy_plugins), the plug-in in your FloPy installation will supersede the plug-in installed as a python package.  Therefore, you will need to delete "flopy_npc_plugin.py" from the "flopy_plugins" folder and remove the import from the "\__init\__.py" file before proceeding.

Once the new FloPy plugin has been installed as a python package, you can create a simulation to test it.  First you will need to restart the kernel for this python notebook, so it is working with a version of python that includes the new FloPy plugin.  Then run the following code to build a simulation.

In [ ]:
import os
from flopy.mf6.modflow.mfgwf import ModflowGwf
from flopy.mf6.modflow.mfgwfdis import ModflowGwfdis
from flopy.mf6.modflow.mfgwfoc import ModflowGwfoc
from flopy.mf6.modflow.mfgwfsto import ModflowGwfsto
from flopy.mf6.modflow.mfims import ModflowIms
from flopy.mf6.modflow.mfsimulation import MFSimulation
from flopy.mf6.modflow.mftdis import ModflowTdis
from flopy.mf6.modflow.mfgwfic import ModflowGwfic
from flopy.mf6.modflow.mfgwfnpf import ModflowGwfnpf
from flopy.mf6.modflow.mfgwfchd import ModflowGwfchd

model_name = "flopy_pkg_test"
pth = "flopy_pkg_test"
# create simulation
sim = MFSimulation(sim_name=model_name, version="mf6", exe_name="mf6.exe", sim_ws=pth)
tdis_package = ModflowTdis(
    sim, time_units="DAYS", nper=1, perioddata=[(1000.0, 120, 1.0)],
    start_date_time="1/1/2010"
)
model = ModflowGwf(
    sim, modelname=model_name, model_nam_file=f"{model_name}.nam"
)
ims_package = ModflowIms(
    sim,
    print_option="SUMMARY",
    complexity="SIMPLE",
    outer_dvclose=0.0001,
    outer_maximum=500,
    under_relaxation="NONE",
    inner_maximum=100,
    inner_dvclose=0.0001,
    rcloserecord=0.001,
    linear_acceleration="CG",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=0.97,
)
sim.register_ims_package(ims_package, [model.name])
bot_data = [-100 for x in range(150)]
dis_package = ModflowGwfdis(
    model,
    nlay=3,
    nrow=15,
    ncol=10,
    delr=100.0,
    delc=100.0,
    top=50.0,
    botm=[5.0, -10.0, {"factor": 1.0, "data": bot_data}],
    filename=f"{model_name}.dis",
)
ic_package = ModflowGwfic(model, strt=50.0, filename=f"{model_name}.ic")
npf_package = ModflowGwfnpf(
    model,
    save_flows=True,
    icelltype=1,
    k=50.0,
    k33=0.1
)
oc_package = ModflowGwfoc(
    model,
    budget_filerecord=f"{model_name}.cbc",
    head_filerecord="AdvGW_tidal.hds",
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)
sto_package = ModflowGwfsto(
    model,
    save_flows=True,
    iconvert=1,
    ss=0.000001,
    sy=0.2,
    transient={0: True, 1: True},
)

chd_period_data = []
for column in range(0, 10):
    chd_period_data.append(((0, 0, column), 50.0))
    chd_period_data.append(((0, 14, column), 10.0))
chd_period = {
    0: chd_period_data
}
chd_package = ModflowGwfchd(
    model,
    print_input=True,
    print_flows=True,
    save_flows=True,
    maxbound=3,
    stress_period_data=chd_period,
)


Add your package to the simulation

In [ ]:
from fp_npc_plugin.mfgwffp_npc import ModflowGwffp_Npc

npc_period = {0: [((2, 0, 0), -200.0)]}
npc_package = ModflowGwffp_Npc(model, maxbound=1, stress_period_data=npc_period)

Write and run the simulation

In [ ]:
sim.write_simulation()
sim.run_simulation()

Look at the model listing file "flopy_pkg_test" and note the "API_NPC_0" package in the model budget.